# Time Series

**Inhalt:** Zeitreihen analysieren Like a Boss

**Nötige Skills:** Daten explorieren, Time+Date Basics

**Lernziele:**
- Datetime-Datentyp anwenden
- Auswertungsmöglichkeiten für zeitbezogene Daten kennenlernen
- Plotting Level 3

**Weitere Ressourcen:**
- https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

## Das Beispiel

Die Zeit ist eine Dimension, die in vielen Datensets vorkommt.

Ein typischer Anwendungsfall sind Finanzdaten. Wir analysieren in diesem Notebook eine einzige Grösse: den Schweizer Aktienindex SMI. Die Daten dazu sind auf täglicher Basis frei verfügbar, ab 1989.

Quelle: https://www.six-group.com/exchanges/indices/data_centre/shares/smi_de.html

Wir werden in diesem Notebook verschiedene Techniken kennenlernen:
- Aus der Gesamtmenge die Zeitperioden auswählen, die uns interessieren
- Daten für verschiedene Zeiträume aggregieren (zB Monate)
- Daten für verschiedene Zeiträume gruppieren (das ist nicht dasselbe)
- Daten-Veränderungen berechnen und darstellen
- Trends erkennen und Daten glätten

Diverse von diesen Techniken kommen üblicherweise zur Anwendungen, wenn man aus einer Zeitreihe eine Story generieren will, und wenn es darum geht, die Daten angemessen darzustellen.

## Vorbereitung

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline

## Daten einlesen

Die Daten sind bereits gespeichert unter `dataprojects/SMI/smi_only.csv`

In [ ]:
df = pd.read_csv('dataprojects/SMI/smi_only.csv', delimiter=';')

In [ ]:
df.head(2)

Wie üblich, Typen abchecken

In [ ]:
df.dtypes

Spalte "High" genauer anschauen

In [ ]:
df['High'].value_counts()

Es hat einige Leerzeichen drin

In [ ]:
df['High'].value_counts().head(1).index

Wir können die Datei nochmals einlesen mit `na_values=`

In [ ]:
df = pd.read_csv('dataprojects/SMI/smi_only.csv', delimiter=';', na_values=' ')

In [ ]:
df.head(2)

In [ ]:
df.dtypes

Nochmals etwas genauer abchecken:

In [ ]:
df.describe()

In der Spalte "Low" gibt es offenbar einige Werte, die nicht stimmen können... (die Börse ging hoffentlich nie auf 0 runter)

In [ ]:
df['Low'].value_counts().head()

Was waren das für Zeitpunkte?

In [ ]:
df[df['Low'] == 0]

Wir korrigieren das... speichern zur Sicherheit nochmals kurz die Index-Zahlen ab

In [ ]:
low_nullen = df[df['Low'] == 0].index

In [ ]:
df.loc[low_nullen]

In [ ]:
df['Low'].replace(0, np.nan, inplace=True)

Check ob es funktioniert hat (wir nutzen die zwischengespeicherten Index-Zahlen)

In [ ]:
df.loc[low_nullen]

In [ ]:
df.describe()

Jetzt sind wir ready! - fast.

## Daten Arrangieren

Bevor es losgeht mit analysieren, erstellen wir uns eine ordentliche Datums-Spalte.

Und zwar eine, die den Datentyp "Datetime" hat!

In [ ]:
df['New Date'] = pd.to_datetime(df['Date'], format="%d.%m.%Y")

In [ ]:
df.head(2)

In [ ]:
df.dtypes

Wir setzen diese Spalte nun als Index

In [ ]:
df.set_index('New Date', inplace=True)

In [ ]:
df.head(2)

Damit das noch etwas schöner aussieht...

In [ ]:
df.rename_axis(None, inplace=True)

In [ ]:
df.head(2)

In [ ]:
df.pop('Date')
df.head(2)

Wir haben nun ein Dataframe mit einer Zeit-formatierten Index-Spalte.

Damit wir die ganze Funktionalität dieses Typus nutzen können, müssen wir die Daten noch zeitlich sortieren.

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.head()

## 1. Slicing

Die erste Technik, die wir kennenlernen, heisst Slicing. Wir schneiden uns also ein bestimmtes Stück aus den Daten heraus. Welches Stück, das geben wir mit einer Kombination aus Jahreszahlen, Monaten, Tagen, ... an.

**Beispiel:** Alle Zeilen mit einem Index-Datum im Jahr 2018

In [ ]:
df['2018'].head()

In [ ]:
df['2018'].tail()

**Beispiel:** Alle Zeilen von 2010 bis 2011

In [ ]:
df['2010':'2011'].head()

In [ ]:
df['2010':'2011'].tail()

**Beispiel:** Alle Zeilen von Januar bis März 2018

In [ ]:
df['2018-01':'2018-03'].head()

In [ ]:
df['2018-01':'2018-03'].tail()

**Beispiel:** Alle Zeilen ab 2017

In [ ]:
df['2017':].head()

In [ ]:
df['2017':].tail()

Easy!

**Mini-Quiz:** Zeigen Sie alle Daten zwischen dem 23. März 2009 und dem 4. April 2009 an.

In [ ]:
#Antwort


## 2. Date-Type-Picking

Die zweite Methode ist: Wir wählen Zeilen nach einem bestimmten Zeit-Kriterium aus - die Zeilen müssen nicht alle am Stück sein, sondern nur eine bestimmte gemeinsame Eigenschaft aufweisen.

Die Zeit-Properties, die wir abfragen können, kennen wir bereits: https://pandas.pydata.org/pandas-docs/stable/timeseries.html

**Beispiel:** Alle ersten Tage des Monats

In [ ]:
df[df.index.day == 1].head()

Hier nochmals eine Liste einiger Properties, nach denen wir filtern können:

| Property | Description |
|----------|------------|
| **`.year`** | - The year of the datetime |
| **`.month`** | - The month of the datetime |
| **`.day`** | - The days of the datetime |
| **`.dayofyear`** | - The ordinal day of year |
| **`.weekofyear`** | - The week ordinal of the year |
| **`.week`** | - The week ordinal of the year |
| **`.dayofweek`** | - The number of the day of the week with Monday=0, Sunday=6 |
| **`.weekday`** | - The number of the day of the week with Monday=0, Sunday=6 |
| **`.weekday_name`** | - The name of the day in a week (ex: Friday) |
| **`.quarter`** | - Quarter of the date: Jan-Mar = 1, Apr-Jun = 2, etc. |
| **`.days_in_month`** | - The number of days in the month of the datetime |
| **`.is_month_start`** | - Logical indicating if first day of month (defined by frequency) |
| **`.is_month_end`** | - Logical indicating if last day of month (defined by frequency) |
| **`.is_quarter_start`** | - Logical indicating if first day of quarter (defined by frequency) |
| **`.is_quarter_end`** | - Logical indicating if last day of quarter (defined by frequency) |
| **`.is_year_start`** | - Logical indicating if first day of year (defined by frequency) |
| **`.is_year_end`** | - Logical indicating if last day of year (defined by frequency) |
| **`.is_leap_year`** | - Logical indicating if the date belongs to a leap year |

**Beispiel:** Alle Tage im April, egal welchen Jahres

In [ ]:
df[df.index.month == 4].head()

**Beispiel:** Alle Montage (die Woche startet mit dem Tag 0!)

In [ ]:
df[df.index.dayofweek == 0].head()

Wir können das auch ohne weiteres plotten

In [ ]:
df[df.index.dayofweek == 0]['Close'].plot(figsize=(8,4))

**Quiz:** Ein Plot mit dem Schlusskurs an allen Freitagen

In [ ]:
#Antwort


**Quiz:** Ein Plot mit dem Schlusskurs an allen Freitagen von 2006 bis 2009

In [ ]:
#Antwort


Ein Problem haben wir allerdings: Wie finden wir den letzten Tag eines Monats, bei dem Daten vorhanden sind?

In [ ]:
df[df.index.is_month_end].head()

## 3. Resampling

Die dritte Technik, die wir kennenlernen, heisst resampling. Wir fassen dabei bestimmte Zeitperioden zusammen. Was wir uns dabei überlegen müssen, ist:
- Welche Zeitperiode uns interessiert
- Welche Zusammenfassungs-Metrik wir bilden wollen
- (und allenfalls: welche Spalte uns interessiert)

**Beispiel:** Der Jahresdurchschnitt der SMI-Kurse. Das heisst also:
- Zeitperiode: jährlich
- Metrik: Durchschnitt
- Spalten: keine spezielle Auswahl

In [ ]:
df.resample('A').mean().head()

Für einen Plot können wir zB eine Spalte speziell auswählen:

In [ ]:
df.resample('A').mean()['Close'].plot(figsize=(8,4))

Hier eine Liste von manchen Zeitperioden, nach denen wir Resamplen können.
(Quelle/volle Liste: http://stackoverflow.com/a/17001474)

| Code | Meaning |
|---------|-----------|
| B       | business day frequency |
| D       | calendar day frequency |
| W       | weekly frequency |
| M       | month end frequency |
| BM      | business month end frequency |
| MS      | month start frequency |
| BMS     | business month start frequency |
| Q       | quarter end frequency |
| BQ      | business quarter endfrequency |
| QS      | quarter start frequency |
| BQS     | business quarter start frequency |
| A       | year end frequency |
| BA      | business year end frequency |
| AS      | year start frequency |
| BAS     | business year start frequency |

**Beispiel: ** Monatliche Mittelwerte

In [ ]:
df.resample('M').mean().head()

**Beispiel: ** Monatliche Mittelwerte, aber mit dem Monats-Startdatum statt mit dem Enddatum

In [ ]:
df.resample('MS').mean().head()

**Beispiel: ** Monatliche Maximalwerte

In [ ]:
df.resample('MS').max().head()

**Beispiel: ** Monatliche Anzahl

In [ ]:
df.resample('MS').count().head()

**Beispiel: ** Monatlicher Letztwert (-> das haben wir vorher gesucht!)

In [ ]:
df.resample('MS').last().head()

**Beispiel:** Wir können das Intervall "Jahr" auch nur in Fünferschritten abfragen

In [ ]:
df.resample('5AS').mean()

**Quiz:** Quartalsweise Minima (mit Startdatum als Index) - Liste

In [ ]:
#Antwort


**Quiz:** Wöchentlicher Median aller Schlusskurse im Jahr 2017 - Plot

In [ ]:
# Antwort


## 4. Grouping

Die vierte Technik, die wir kennenlernen, kennen wir eigentlich bereits: Groupby.

Hier geht es um zyklische Analysen, also zB Zusammenfassungen davon, was jeweils im Januar (egal welchen Jahres) passiert ist.

Was wir uns wiederum überlegen müssen, ist:
- Welcher Zeitintervall interessiert uns
- Welche Metrik wollen wir anwenden
- (und auf welche Spalte wollen wir das einschränken)

Technisch gesehen gruppieren wir die Daten eigentlich immer nach bestimmten, repetitiven Werten, die wir dem Index entnehmen:

In [ ]:
df.index.month

Wir gruppieren die Daten also zB nach den Zahlen 1, 2, 3, .... 12, wenn wir nach Monat gruppieren

In [ ]:
df.groupby(df.index.month).mean()

Wir können die gruppierte Auswertung wiederum plotten, wobei wir hier keine Line- sondern Barcharts verwenden sollten!

(das Ergebnis sieht jetzt nicht so spektakulär aus, aber das ändern wir später noch!)

In [ ]:
df.groupby(df.index.month)['Close'].mean().plot(figsize=(8,4), kind='bar')

**Beispiel:** Was war der höchste Börsenkurs, der je an einem Montag, Dienstag, ... Freitag erzielt wurde? Liste

In [ ]:
df.groupby(df.index.dayofweek)['High'].max()

**Quiz:** Ein Jahr hat 52 Wochen. Gibt es ein typisches Muster, wie sich die mittleren Schlusskurse entwickeln? Plot

In [ ]:
#Antwort


## 6. Percent change

Die Fragen, die wir soeben gestellt haben, werden einiges lustiger zu beantworten, wenn wir nicht die absoluten Kurse vergleichen (zB je nach Monat), sondern die relativen Veränderungen.

Dafür ist `pct_change()` da: Eine Funktion, die man auf das ganze Dataframe anwenden kann, um die täglichen, prozentualen Veränderungen auszurechen.

Oder man kann `pct_change()` auch auf Daten anwenden, die man bereits resamplet hat!

**Beispiel:** Wie verändert sich die Börse typischerweise über die Jahresmonate hinweg?

Wir starten dazu mit den monatlichen Mittelwerten für den ganzen Zeitraum, die wir mittels `resample()` bereits einmal ermittelt haben:

In [ ]:
df.resample('MS').mean().head()

Nun berechnen wir die monatlichen Veränderungsraten (warum ist die erste Spalte = NaN?)

In [ ]:
df_mth_chg = df.resample('MS').mean().pct_change()

In [ ]:
df_mth_chg.head()

Letzter Schritt: Wir gruppieren die monatlichen Veränderungsraten - nach Monaten. Und mitteln.

In [ ]:
df_mth_chg.groupby(df_mth_chg.index.month).mean()

Im Plot der Schlusskurse sieht das dann so aus:

In [ ]:
df_mth_chg.groupby(df_mth_chg.index.month)['Close'].mean().plot(figsize=(8,4), kind='bar')

"Sell in May and go away", lautet eine Börsenweisheit. Stimmt sie?

## 6. Rolling

Die sechste Technik, wie wir die Daten modifizieren können, ist: Rollen.

Wir schauen diese Technik nicht genauer an. Es geht darum:
- über ein bestimmtes Zeitfenster (zB 10 Tage)
- eine bestimmte Metrik anzuwenden
- (auf eine Spalte, falls wir wollen)

Hier wäre ein Anwendungsbeispiel: der gleitende ("rollende") Zehn-Tages-Durchschnitt der Börsenkurse. Wir müssen jeweils angeben, wie das Zeitfenster positioniert werden soll (zentriert oder nicht).

Mehr Infos: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html

In [ ]:
df.rolling(10, center=True).mean().head(20)

# Plotting Level 3

Hier lernen wir:
- Einen Befehl, der ein paar Codezeilen spart: `agg()`
- Erstmals eine Funktion aus Matplotlib direkt anzusteuern: `fill_between()`

**Aggregate**

Wir haben bereits kennengelernt, wie wir mehrere Linien auf einen Plot zeichnen können.

In diesem Fall geht das eigentlich ganz einfach: Wir schreiben einfach alle `plot()`-Befehle in die Zelle.

Zum Beispiel: Die monatlichen Maximal-, Minimal- und Durchschnittskurse (seit 2014).

In [ ]:
df['2014':].resample('MS').min()['Close'].plot(figsize=(8,4))
df['2014':].resample('MS').max()['Close'].plot()
df['2014':].resample('MS').mean()['Close'].plot()

Wir können die drei Metriken `min()`, `max()` und `mean()` aber auch in einen Befehl packen:

In [ ]:
df['2014':].resample('MS').agg(['min', 'max', 'mean'])['Close'].plot(figsize=(8,4))

Ohne Plot sehen wir, was dahinter steht:

In [ ]:
df['2014':].resample('MS').agg(['min', 'max', 'mean'])['Close'].head()

**Fill Between**

Bislang haben wir Plots stets mit der `plot()`-Funktion erstellt. Diese Funktion ist von verschiedenen Pandas-Objekttypen wie Series oder Dataframes her aufrufbar.

Die Matplotlib-Bibliothek bietet aber auch von sich aus Plotting-Funktionen an. Sie gehen zT weiter und erlauben uns mehr Möglichkeiten, Plots zu gestalten.

Um sie zu benutzen, müssen wir sie aber importieren. Konvention: als `plt`

In [ ]:
#Brauchen wir zum plotten
import matplotlib.pyplot as plt

Eine Funktion, die wir aufrufen können, heisst: `fill_between()`. Sie benötigt drei Inputs:
- Eine Liste von x-Werten
- Eine Liste von y-Maximalwerten
- Eine Liste von y-Minimalwerten

Wir stellen diese Werte mal zusammen: Es sind dieselben wie bereits in der vorherigen Grafik.

In [ ]:
x_values = df['2014':].resample('MS')['Close'].mean().index
y_mins = df['2014':].resample('MS')['Close'].min()
y_maxes = df['2014':].resample('MS')['Close'].max()

Dann füllen wir die Werte in die Funktion ein:

In [ ]:
plt.fill_between(x_values, y_mins, y_maxes, alpha=0.5)

Wenn wir möchten, können wir zusätzlich noch die mittleren Werte dazuzeichnen:

In [ ]:
df['2014':].resample('MS').mean()['Close'].plot(figsize=(8,4))
plt.fill_between(x_values, y_mins, y_maxes, alpha=0.5)

# Übung

Wir waren bisher fast immer an den täglichen Schlusskursen interessiert. Nun schauen wir uns mal die täglichen Schwankungen an.

Wir bilden eine neue Spalte: Range = Differenz zwischen Tageshoch und -tief, in Prozent, gemessen am Schlusskurs.

In [ ]:
df['Range'] = (df['High'] - df['Low']) / df['Close'] * 100

In [ ]:
df.head(5)

### Beginner

Ein paar Fragen zum Aufwärmen.

Erstellen Sie einen einfachen Plot mit den täglichen Schwankungswerten, für den gesamten Zeitraum.

Sieht das in Ordnung aus? Wählen Sie einen besseren Zeitraum, zB ab 1994:

Was sehen Sie auf dem Bild? Gab es Phasen, in denen die Schwankungen grösser waren?

In [ ]:
# Antwort als Text
# 

Stellen Sie denselben Chart dar, aber mit Schwankungen im Wochenmittel.

... und im Monatsmittel ...

... und im Quartalssmittel.

Welcher der vorherigen Charts sagt am meisten aus? Warum?

In [ ]:
# Antwort in Worten
# 

Plotten Sie die Tagesschwankung an allen Freitagen (immer noch ab 1994).

In [ ]:
df['1994':][df['1994':].index.weekday == 4]['Range'].plot(figsize=(8,4))

Vergleichen Sie diesen Chart mit dem Chart der Wochen-Durchschnittswerte. Welchen würden Sie verwenden? Warum?

In [ ]:
# Antwort in Worten
# 

### Advanced

Unterscheiden sich die Schwankungen je nach Monat? Plot, Daten ab 1994.

In [ ]:
# Antwort in Worten
# 

Steht die Höhe der Schwankungen in Zusammenhang mit dem Kursstand? Scatterplot, mit sehr kleinen Punkten.

In [ ]:
# Antwort in Worten
# 
# 

Erstellen Sie eine Rangliste der zehn Tage, an denen es die grössten Schwankungen gab. (ab 1994)

Dieselbe Rangliste, aber mit den Top 100. Plotten Sie diese auf einem Punkte-Diagramm:
- x: Zeitachse
- y: Höhe des Kurses.

Tipp: Attribute `linestyle=`, `marker=`, `markersize=` verwenden.
- Siehe https://stackoverflow.com/questions/8409095/matplotlib-set-markers-for-individual-points-on-a-line

Ihr Eindruck dieses Charts?

In [ ]:
# Antwort in Worten
# 

### Pro

Kreieren Sie drei Zeitreihen:
- Monatliche Serie von Datumsangaben: 1994-01-01, 1994-02-01, etc.
- Monatliches Schwankungsmaximum seit 1994
- Monatliches Schwankungsminimum seit 1994

Zeichnen Sie einen Fill-Between-Chart mit den drei Zeitreihen

Zeichnen Sie zusätzlich auf dem Chart eine Linie mit den monatlichen Schwankungs-Mittelwerten ein.